In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value,Mato Grosso - Consumo de Cimento (t)
0,2003-1,50.917240,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.297178,0.745147,6.084564e+07,2.670588e+06,16.831332,5.118946e+07,0.331800,47.470
1,2003-2,50.851586,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.291317,0.745308,6.092768e+07,2.672688e+06,16.835967,5.121614e+07,0.333615,45.387
2,2003-3,50.970549,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.285457,0.745469,6.100973e+07,2.674788e+06,16.840602,5.124282e+07,0.334864,44.907
3,2003-4,51.049978,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.279596,0.745630,6.109177e+07,2.676888e+06,16.845237,5.126950e+07,0.336048,45.467
4,2003-5,51.567529,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.273735,0.745791,6.117381e+07,2.678988e+06,16.849872,5.129619e+07,0.336447,56.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,119.544326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600607,192.533
236,2022-9,118.223448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.598965,183.895
237,2022-10,117.524152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596866,180.525
238,2022-11,116.430559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.594246,161.683


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      51.972
1      40.729
2      54.446
3      51.788
4      58.702
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.588452,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.621493,0.994318,0.973043,0.688394,0.986108,0.975683,0.938508
158,0.582813,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.596535,0.988636,0.974495,0.684427,0.985875,0.976460,0.931775
159,0.576827,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.571578,0.982954,0.975947,0.680460,0.985641,0.977238,0.924689
160,0.568447,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.546620,0.977272,0.977399,0.676493,0.985408,0.978015,0.918022


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       51.972
1       40.729
2       54.446
3       51.788
4       58.702
        ...   
157     65.293
158     90.338
159     83.188
160    107.230
161    105.397
Name: Mato Grosso - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
126,1.000000,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.366310,0.798772,0.815216,0.971029,0.870123,0.843018,0.868231
127,0.991476,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.388500,0.797810,0.822368,0.966201,0.876689,0.849696,0.879874
128,0.976090,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.410690,0.796849,0.829519,0.961372,0.883254,0.856374,0.890140
129,0.962811,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.432879,0.795888,0.836671,0.956544,0.889819,0.863052,0.900472
130,0.946237,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.455069,0.794926,0.843822,0.951716,0.896384,0.869730,0.912345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1719479622, 3013794340, 4275837557, 383586916, 3371642246, 3573270711, 4282409986, 73237738, 1921382146, 141561786, 1911106934, 1303695431, 3499041090, 3264438077, 515574670, 1021877548, 3200841210, 3794557442, 533834484, 673842339, 4113175143, 2872753359, 2608002709, 2464198575, 2807599551, 1709097582, 3222455832, 1632090359, 2522314039, 2860252237, 3890213589, 3594358859, 426060538, 1279844865, 3227336353, 2810193439, 762627949, 1916038592, 3176425093, 115862435, 695583839, 2877020805, 779251982, 334760895, 1789613609, 1784751688, 3917190150, 786016214, 4173695063, 3019176347]


Step: 0 ___________________________________________
val_loss: 77.32479095458984
winner_seed: 1719479622


Step: 1 ___________________________________________
val_loss: 63.4732666015625
winner_seed: 3013794340


Step: 2 ___________________________________________
val_loss: 92.3858871459961


Step: 3 ___________________________________________
val_loss: 57.80889892578125
winner_seed: 383586916


Step: 4 ______

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 282ms/step - loss: 7403.3623 - val_loss: 831.2336
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 673.8102 - val_loss: 455.3294
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 503.6784 - val_loss: 509.4312
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 517.4417 - val_loss: 450.5114
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 492.7687 - val_loss: 443.8699
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 490.9907 - val_loss: 438.0307
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 497.0232 - val_loss: 434.5442
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 478.3389 - val_loss: 434.0396
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 489.2970 - val_loss: 439.4046
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step - l

4/4 [==============================] - 0s 9ms/step - loss: 217.6658 - val_loss: 195.5232
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 208.3671 - val_loss: 204.1373
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 297.8167 - val_loss: 201.8707
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 224.3349 - val_loss: 226.4408
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 222.7792 - val_loss: 441.3784
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 500.1449 - val_loss: 233.5415
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 235.1996 - val_loss: 175.7592
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 224.5416 - val_loss: 225.7069
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 200.8817 - val_loss: 172.7939
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 201.

4/4 [==============================] - 0s 10ms/step - loss: 161.8298 - val_loss: 163.8888
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 248.6905 - val_loss: 150.4616
Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 186.1685 - val_loss: 168.7246
Epoch 160/10000
4/4 [==============================] - 0s 8ms/step - loss: 190.6934 - val_loss: 156.0099
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 165.2077 - val_loss: 240.2443
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 211.2744 - val_loss: 153.2679
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 235.4141 - val_loss: 135.1862
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 173.2897 - val_loss: 151.5717
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 184.1141 - val_loss: 168.9399
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step -

4/4 [==============================] - 0s 9ms/step - loss: 191.1994 - val_loss: 167.4679
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 173.1021 - val_loss: 153.3270
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 213.1343 - val_loss: 226.5556
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 200.1083 - val_loss: 195.7224
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 198.8308 - val_loss: 178.8180
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 186.6114 - val_loss: 237.4409
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 258.6844 - val_loss: 254.6875
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 172.6829 - val_loss: 128.7693
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 155.8519 - val_loss: 116.9770
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step -

Epoch 313/10000
4/4 [==============================] - 0s 9ms/step - loss: 241.8300 - val_loss: 178.3347
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 182.1706 - val_loss: 110.4995
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 211.8075 - val_loss: 179.0036
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 238.9887 - val_loss: 135.9407
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 162.3710 - val_loss: 222.1570
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 226.9680 - val_loss: 147.8624
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 157.2320 - val_loss: 134.0665
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 169.5975 - val_loss: 200.9012
Epoch 321/10000
4/4 [==============================] - 0s 10ms/step - loss: 175.4943 - val_loss: 123.1198
Epoch 322/10000
4/4 [==============================]

...

4/4 [==============================] - 0s 9ms/step - loss: 125.2619 - val_loss: 198.1733
Epoch 859/10000
4/4 [==============================] - 0s 9ms/step - loss: 111.1029 - val_loss: 136.7952
Epoch 860/10000
4/4 [==============================] - 0s 9ms/step - loss: 103.9512 - val_loss: 213.8112
Epoch 861/10000
4/4 [==============================] - 0s 9ms/step - loss: 116.5233 - val_loss: 190.7352
Epoch 862/10000
4/4 [==============================] - 0s 8ms/step - loss: 107.6816 - val_loss: 165.4465
Epoch 863/10000
4/4 [==============================] - 0s 8ms/step - loss: 109.8935 - val_loss: 200.3537
Epoch 864/10000
4/4 [==============================] - 0s 8ms/step - loss: 105.5207 - val_loss: 194.8732
Epoch 865/10000
4/4 [==============================] - 0s 9ms/step - loss: 146.6635 - val_loss: 197.6932
Epoch 866/10000
4/4 [==============================] - 0s 10ms/step - loss: 178.8868 - val_loss: 162.4156
Epoch 867/10000
4/4 [==============================] - 0s 9ms/step - l

4/4 [==============================] - 0s 9ms/step - loss: 154.2969 - val_loss: 102.3070
Epoch 937/10000
4/4 [==============================] - 0s 10ms/step - loss: 148.6443 - val_loss: 182.6958
Epoch 938/10000
4/4 [==============================] - 0s 9ms/step - loss: 122.7052 - val_loss: 107.8075
Epoch 939/10000
4/4 [==============================] - 0s 8ms/step - loss: 131.1513 - val_loss: 119.8459
Epoch 940/10000
4/4 [==============================] - 0s 8ms/step - loss: 92.9123 - val_loss: 136.0536
Epoch 941/10000
4/4 [==============================] - 0s 8ms/step - loss: 74.7991 - val_loss: 164.9902
Epoch 942/10000
4/4 [==============================] - 0s 10ms/step - loss: 76.8878 - val_loss: 153.8070
Epoch 943/10000
4/4 [==============================] - 0s 9ms/step - loss: 97.5419 - val_loss: 211.2732
Epoch 944/10000
4/4 [==============================] - 0s 10ms/step - loss: 107.3646 - val_loss: 159.0184
Epoch 945/10000
4/4 [==============================] - 0s 9ms/step - los

4/4 [==============================] - 0s 9ms/step - loss: 69.9022 - val_loss: 169.9059
Epoch 1015/10000
4/4 [==============================] - 0s 9ms/step - loss: 75.0203 - val_loss: 162.5035
Epoch 1016/10000
4/4 [==============================] - 0s 9ms/step - loss: 55.2065 - val_loss: 156.7627
Epoch 1017/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.3150 - val_loss: 205.9842
Epoch 1018/10000
4/4 [==============================] - 0s 9ms/step - loss: 58.9771 - val_loss: 203.8793
Epoch 1019/10000
4/4 [==============================] - 0s 8ms/step - loss: 68.0256 - val_loss: 196.3310
Epoch 1020/10000
4/4 [==============================] - 0s 8ms/step - loss: 83.0410 - val_loss: 232.0625
Epoch 1021/10000
4/4 [==============================] - 0s 10ms/step - loss: 74.3907 - val_loss: 185.7251
Epoch 1022/10000
4/4 [==============================] - 0s 10ms/step - loss: 67.0615 - val_loss: 144.1176
Epoch 1023/10000
4/4 [==============================] - 0s 9ms/step - 

Epoch 1092/10000
4/4 [==============================] - 0s 8ms/step - loss: 60.1993 - val_loss: 138.5204
Epoch 1093/10000
4/4 [==============================] - 0s 9ms/step - loss: 66.1103 - val_loss: 195.5092
Epoch 1094/10000
4/4 [==============================] - 0s 8ms/step - loss: 63.4689 - val_loss: 174.8297
Epoch 1095/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.5184 - val_loss: 143.2348
Epoch 1096/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.9240 - val_loss: 167.0104
Epoch 1097/10000
4/4 [==============================] - 0s 11ms/step - loss: 51.7065 - val_loss: 204.0026
Epoch 1098/10000
4/4 [==============================] - 0s 9ms/step - loss: 58.5131 - val_loss: 168.0707
Epoch 1099/10000
4/4 [==============================] - 0s 10ms/step - loss: 56.2961 - val_loss: 158.0217
Epoch 1100/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.7650 - val_loss: 176.6197
Epoch 1101/10000
4/4 [==============================]

Epoch 1170/10000
4/4 [==============================] - 0s 9ms/step - loss: 40.1794 - val_loss: 126.5138
Epoch 1171/10000
4/4 [==============================] - 0s 9ms/step - loss: 41.0775 - val_loss: 131.4466
Epoch 1172/10000
4/4 [==============================] - 0s 9ms/step - loss: 39.5839 - val_loss: 154.3902
Epoch 1173/10000
4/4 [==============================] - 0s 9ms/step - loss: 37.8386 - val_loss: 164.8466
Epoch 1174/10000
4/4 [==============================] - 0s 9ms/step - loss: 45.4634 - val_loss: 142.0032
Epoch 1175/10000
4/4 [==============================] - 0s 9ms/step - loss: 40.6462 - val_loss: 147.1178
Epoch 1176/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.2895 - val_loss: 151.1917
Epoch 1177/10000
4/4 [==============================] - 0s 9ms/step - loss: 55.6360 - val_loss: 131.7371
Epoch 1178/10000
4/4 [==============================] - 0s 8ms/step - loss: 38.5347 - val_loss: 188.1579
Epoch 1179/10000
4/4 [==============================] -

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [42]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,95.930099,89.598854,86.451584,85.180969,85.192024,86.980156,90.545868,96.675201,102.399986,106.788254,109.965096,112.668945,111.009773,112.008751,112.763977,111.072174,108.165421,103.484642,101.567963,101.04689,101.865425,104.182755,107.720703,111.541077,114.428558,115.746033,113.414581,108.340027,102.669899,97.002327
Target,120.055,125.769,112.904,123.201,112.725,90.828,96.36,86.444,94.285,98.986,88.072,141.01,122.652,142.145,121.124,130.503,104.115,90.69,102.685,96.144,102.197,106.712,124.057,124.625,133.116,144.31,140.357,152.769,124.038,95.054
Error,24.124901,36.170143,26.452415,38.020027,27.532974,3.847847,5.814133,10.231201,8.114983,7.802254,21.893097,28.341049,11.642227,30.136253,8.360023,19.430832,4.050423,12.79464,1.117035,4.902893,0.331573,2.529243,16.336296,13.083923,18.687439,28.563965,26.942413,44.42897,21.368103,1.948326


In [45]:
display(mae)
display(mape)

16.83332

0.13851282

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1148.377]] - Target[1290.639]| =  Error: [[142.26208]]; MAPE:[[0.11022608]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1286.4294]] - Target[1367.6490000000001]| =  Error: [[81.219604]]; MAPE:[[0.05938629]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[651.60144]] - Target[789.644]| =  Error: [[138.04254]]; MAPE:[[0.17481618]]


[array([[142.26208]], dtype=float32),
 array([[81.219604]], dtype=float32),
 array([[138.04254]], dtype=float32)]

120.50808

0.11480951